# Exercice SQLAlchemy
- create database "from_python_DB" via SSMS
- create tables "iris_nopandas" and "iris_pandas"
- insert iris data (from seaborn) into those tables:
  1. manual CRUD into "iris_nopandas"
  2. via pandas into "iris_pandas"
- select all data from iris table from "iris_pandas" in both ways:
  1. into a pyhton list (no pandas) 
  2. into a dataframe (via pandas)

In [1]:
from seaborn import load_dataset

iris = load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# Some mandatory imports

In [ ]:
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine, text

# Connecting Database 

In [ ]:
### ====== VARIABLES ====== ###
dbms = "mssql+pyodbc"
driver = "ODBC Driver 17 for SQL Server"
server = r"PORTABLE_TOM\DATAVIZ"
database = "from_python_DB"


try:
    # Connexion à SQL Server
    conn = pyodbc.connect(
        f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
    )
    cursor = conn.cursor()

    # Exécuter une requête simple pour tester
    cursor.execute("SELECT @@VERSION")
    row = cursor.fetchone()

    print("Connexion réussie :", row[0])

except Exception as e:
    print("Erreur de connexion :", e)


con_string_trusted_windows_connection = (
    f"{dbms}://{server}/{database}?trusted_connection=yes&driver={driver}"
)

### fully configured connection
engine = create_engine(con_string_trusted_windows_connection)

Connexion réussie : Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 22631: ) (Hypervisor)



# Using engine

In [ ]:
open_connection = engine.connect()

result_query = open_connection.execute(text("""SELECT 'Hello World'"""))
### result_query = reference/temporary value
### "text"-function required for SQL alchemy: when feeding text-based queries, alternatives= SQLAlchemy-object
res = (list(result_query)).copy()  ## independent copy

open_connection.close()  ### need to close the file/connection when done
print(list(result_query))  ### connection closed, now referenced to empty value
print(res)  ### copied values, stayed

[]
[('Hello World',)]


Now lets create some database configurations, creating tables:
-  iris_nopandas 
-  iris_pandas

In [ ]:
CREATE_TABLE_NO_PANDAS = """
    CREATE TABLE iris_nopandas (
        id INT IDENTITY(1,1) PRIMARY KEY,
        sepal_length FLOAT,
        sepal_width FLOAT,
        petal_length FLOAT,
        petal_width FLOAT,
        species VARCHAR(50)
)
"""

CREATE_TABLE_PANDAS = """
    CREATE TABLE iris_pandas (
        id INT IDENTITY(1,1) PRIMARY KEY,
        sepal_length FLOAT,
        sepal_width FLOAT,
        petal_length FLOAT,
        petal_width FLOAT,
        species VARCHAR(50)
)
"""
DROP_TABLE = "DROP TABLE iris_pandas"
DROP_TABLE_NOPANDAS = "DROP TABLE iris_nopandas"

In [ ]:
with engine.connect() as con:
    try:
        con.execute(text(DROP_TABLE))
        con.execute(text(DROP_TABLE_NOPANDAS))
        con.execute(text(CREATE_TABLE_NO_PANDAS))
        con.execute(text(CREATE_TABLE_PANDAS))
        print("Tables created successfully")
    except Exception as e:
        print("Error creating tables:", e)
    finally:
        con.commit()

Tables created successfully


# Insert iris data into (no_)panda table

In [ ]:
block_insert = """
                    INSERT INTO iris_pandas (sepal_length, sepal_width, petal_length, petal_width, species) VALUES 
                    (:sepal_length, :sepal_width, :petal_length, :petal_width, :species)
                """

list_of_dictionaries = iris.to_dict("records")

with engine.connect() as conn:
    ### block insert
    try:
        conn.execute(text(block_insert), list_of_dictionaries)
        print("Data inserted successfully")
    except Exception as e:
        print("Error inserting data:", e)
    finally:
        conn.commit()

Data inserted successfully


- select all data from iris table from "iris_pandas" in both ways:
  1. into a pyhton list (no pandas) 
  2. into a dataframe (via pandas)

First using no_pandas

In [ ]:
with engine.connect() as conn:
    try:
        result = conn.execute(text("SELECT * FROM iris_nopandas"))
        res = list(result).copy()
    except Exception as e:
        print("Error fetching data:", e)
    finally:
        conn.commit()
print(res)

[]


# Now using pandas

In [ ]:
with engine.connect() as conn:
    try:
        res2 = pd.read_sql_table("iris_pandas", conn)

    except Exception as e:
        print("Error fetching data:", e)
    finally:
        conn.commit()
display(res2.head(5))

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa
